In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import sys
sys.path.append('/Volumes/KT-TOSHIBA/ENOI/CODE')
import enoi_functions as enoi

mdata='/Volumes/KT-TOSHIBA/ENOI/sorted_model_data/'

In [2]:
from dask.distributed import Client
client = Client('tcp://127.0.0.1:8786')

In [3]:
ul=0
ll=100

In [4]:
dissic_ll = np.load(f'{mdata}dic_{ll}.npy')
thetao_ll = np.load(f'{mdata}thetao_{ll}.npy')
so_ll = np.load(f'{mdata}so_{ll}.npy')

drift_dissic_ll = np.load(f"drift_ts_dissic{ll:04d}.npy")
drift_thetao_ll = np.load(f"drift_ts_thetao{ll:04d}.npy")
drift_so_ll = np.load(f"drift_ts_so{ll:04d}.npy")

if ul > 0:
    dissic_ul = np.load(f'{mdata}dic_{ul}.npy')
    thetao_ul = np.load(f'{mdata}thetao_{ul}.npy')
    so_ul = np.load(f'{mdata}so_{ul}.npy')

    drift_dissic_ul = np.load(f"drift_ts_dissic{ul:04d}.npy")
    drift_thetao_ul = np.load(f"drift_ts_thetao{ul:04d}.npy")
    drift_so_ul = np.load(f"drift_ts_so{ul:04d}.npy")

    ens_dic =  xr.DataArray(data = dissic_ll - dissic_ul - (drift_dissic_ll - drift_dissic_ul))
    ens_theta =  xr.DataArray(data = thetao_ll - thetao_ul - (drift_thetao_ll - drift_thetao_ul))
    ens_sal = xr.DataArray(data = so_ll - so_ul - (drift_so_ll - drift_so_ul))
elif ul == 0:
    ens_dic =  xr.DataArray(data = dissic_ll  - drift_dissic_ll)
    ens_theta =  xr.DataArray(data = thetao_ll - drift_thetao_ll)
    ens_sal = xr.DataArray(data = so_ll - drift_so_ll)

In [8]:
argo_coverage= xr.open_dataset('/Volumes/KT-TOSHIBA/ENOI/CODE/argo_mask.nc')
argo_mask = argo_coverage.argo_annualcoverage

years_used = -60
limit_z = 100
m0 = xr.open_dataset('/Volumes/KT-TOSHIBA/ENOI/CMIP6/forcings/mol1.nc', decode_times=False)
pco2 = m0.mole_fraction_of_carbon_dioxide_in_air[years_used:,0]
pco2_extended = np.tile(pco2.data - np.nanmean(pco2.data), 
                        int(np.shape(so_ll)[0]/np.abs(years_used)))

lon = np.arange(0.5,360.5)
lat = np.arange(-89.5,90.5)

mdict = {'lat':'lat', 'lon':'lon', 'lev':'lev', 'lev_bnds':'lev_bnds'}

##############################################################################
# Creating masks to plot figures
interp_mask, land_mask = enoi.create_mask(dissic_ll, f'{mdata}mask_{ll}.npy')
newcmp = enoi.n_cmap()
##############################################################################

In [9]:
dic_resid = ens_dic - ens_dic.mean(axis=0)
theta_resid = ens_theta - ens_theta.mean(axis=0)
so_resid = ens_sal - ens_sal.mean(axis=0)

In [10]:
coeff_ensCO2 = np.zeros([180,360])
coeff_ensT = np.zeros([180,360])
coeff_ensS = np.zeros([180,360])
for i in np.arange(0,180):
        for j in np.arange(0,360):
            if theta_resid[0,i,j] != 0 :
                input_vars = np.stack((pco2_extended, theta_resid[:,i,j],ens_sal[:,i,j]), axis=-1)
                dummy = np.linalg.lstsq(input_vars, dic_resid[:,i,j], rcond=-1)
                
                coeff_ensCO2[i,j] = dummy[0][0]
                coeff_ensT[i,j] = dummy[0][1]
                coeff_ensS[i,j] = dummy[0][2]

In [11]:
ds1 = xr.Dataset(
      data_vars=dict(
          cCO2=(["y", "x"], coeff_ensCO2.astype('float64')),
          cT=(["y", "x"], coeff_ensT.astype('float64')),
          cS=(["y", "x"], coeff_ensS.astype('float64'))
          ),

      coords=dict(
          lon=(["x"], lon),
          lat=(["y"], lat),
          ),

      attrs=dict(description=f"Co-located coefficients for ENOI, DIC {ul}m-{ll}m integrated"),

  )

output_fn = f'c_{ul}_{ll}.nc'
    
print(f'Saving in {output_fn}')
ds1.to_netcdf(output_fn)

Saving in c_0_100.nc


In [12]:
coeff_ensCO2_omit = np.zeros([6,180,360])
coeff_ensT_omit = np.zeros([6,180,360])
coeff_ensS_omit = np.zeros([6,180,360])
for k in np.arange(6):
    ri_i = np.arange(k*5*60,(k+1)*5*60)
    full = np.arange(30*60)
    test_index = np.delete(full,ri_i)
    
    for i in np.arange(0,180):
        for j in np.arange(0,360):
            if theta_resid[0,i,j] != 0 :
                input_vars = np.stack((pco2_extended[test_index], theta_resid[test_index,i,j],so_resid[test_index,i,j]), axis=-1)
                dummy = np.linalg.lstsq(input_vars, dic_resid[test_index,i,j], rcond=-1)
                
                coeff_ensCO2_omit[k,i,j] = dummy[0][0]
                coeff_ensT_omit[k,i,j] = dummy[0][1]
                coeff_ensS_omit[k,i,j] = dummy[0][2]
    print(k)

mod=["ACCESS", "CanESM", "CESM", "IPSL", "MPI", "UKESM"]

ds1 = xr.Dataset(
     data_vars=dict(
         cCO2=(["mod", "y", "x"], coeff_ensCO2_omit.astype('float64')),
         cT=(["mod", "y", "x"], coeff_ensT_omit.astype('float64')),
         cS=(["mod", "y", "x"], coeff_ensS_omit.astype('float64'))
         ),

     coords=dict(
         model_omitted=(["mod"], mod),
         lon=(["x"], lon),
         lat=(["y"], lat),
         ),

     attrs=dict(description=f"Co-located coefficients for ENOI, DIC {ul}m-{ll}m integrated - sensitivity experiments"),

 )

output_fn = f'c_{ul}_{ll}_sens.nc'
    
print(f'Saving in {output_fn}')
ds1.to_netcdf(output_fn)

0
1
2
3
4
5
Saving in c_0_100_sens.nc


distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
